In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
crop_df = pd.read_csv(r'D:\ECO221_Group-25\data\ECO221_Project_2025_Final.csv')

In [6]:
rainfall_df = pd.read_csv(r'D:\ECO221_Group-25\data\RF_DistrictWise_ECO221_2025.csv')

In [9]:
salinity_df = pd.read_csv(r'D:\ECO221_Group-25\data\Salinity_Alkalinity_ECO221_2025.csv')

In [10]:
sugarcane_df = crop_df[crop_df["crop"].str.lower()=="sugarcane"].copy()

In [11]:
#Capping the irrigated Area
sugarcane_df["irrigatedarea_capped"] = sugarcane_df[["irrigatedarea1000hectares","area1000hectares"]].min(axis=1)

In [13]:
sugarcane_df["irrigatedarea_new"] = sugarcane_df["irrigatedarea_capped"] + 1


In [14]:
sugarcane_df["unirrigated_area"] = sugarcane_df["area1000hectares"] - sugarcane_df["irrigatedarea_capped"]

In [15]:
sugarcane_df.isnull().sum()

crop                           0
area1000hectares              65
production1000tonnes          65
irrigatedarea1000hectares     65
districtcode                   0
year                           0
statecode                      0
statename                      0
nitrogenconsumptiontonnes      0
phosphateconsumptiontonnes     0
potashconsumptiontonnes        0
districtname                   0
irrigatedarea_capped          65
irrigatedarea_new             65
unirrigated_area              65
dtype: int64

In [16]:
rainfall_2017 = rainfall_df[rainfall_df['yearcode'] == 2017]

In [17]:
rainfall_summary = rainfall_2017.groupby('district')['rf'].sum().reset_index()

In [18]:
rainfall_summary.columns = ['districtname','annual_rainfall']

In [19]:
salinity_df.columns = ['districtname','salinity_percent']

In [20]:
sugarcane_df['districtname'] = sugarcane_df['districtname'].str.strip().str.lower()
rainfall_summary['districtname'] = rainfall_summary['districtname'].str.strip().str.lower()
salinity_df['districtname'] = salinity_df['districtname'].str.strip().str.lower()

In [21]:
print("Common districts (rainfall):", set(sugarcane_df['districtname']) & set(rainfall_summary['districtname']))
print("Common districts (salinity):", set(sugarcane_df['districtname']) & set(salinity_df['districtname']))

Common districts (rainfall): {'raichur', 'hyderabad', 'sangli', 'palakkad', 'jalaun', 'budaun', 'uttara kannada', 'surendranagar', 'kheri', 'kurnool', 'fatehpur', 'aurangabad', 'meerut', 'kota', 'jaipur', 'raisen', 'solan', 'chittorgarh', 'bhavnagar', 'lucknow', 'ratnagiri', 'bhind', 'west godavari', 'sidhi', 'krishna', 'deoria', 'bharuch', 'amreli', 'shahdol', 'pratapgarh', 'malappuram', 'barabanki', 'banswara', 'hardoi', 'unnao', 'satna', 'saharsa', 'madurai', 'vidisha', 'muzaffarpur', 'muzaffarnagar', 'nalgonda', 'jhabua', 'nagpur', 'kangra', 'beed', 'sultanpur', 'darrang', 'pune', 'chhatarpur', 'chhindwara', 'kottayam', 'rohtak', 'uttar kashi', 'akola', 'aligarh', 'ahmednagar', 'jalore', 'sangrur', 'lakhimpur', 'osmanabad', 'rajkot', 'thrissur', 'chamba', 'jabalpur', 'gurdaspur', 'rewa', 'mahabubnagar', 'ludhiana', 'chandrapur', 'goalpara', 'nagaon', 'ambala', 'thiruvananthapuram', 'thanjavur', 'khammam', 'tikamgarh', 'bhilwara', 'ganganagar', 'varanasi', 'kinnaur', 'raigad', 'hosh

In [22]:
print("Sugarcane districts:\n", sugarcane_df['districtname'].dropna().unique()[:10])
print("\nRainfall districts:\n", rainfall_summary['districtname'].dropna().unique()[:10])
print("\nSalinity districts:\n", salinity_df['districtname'].dropna().unique()[:10])

Sugarcane districts:
 ['durg' 'bastar' 'raipur' 'bilaspur' 'raigarh' 'surguja' 'jabalpur'
 'balaghat' 'chhindwara' 'narsinghpur']

Rainfall districts:
 ['24 paraganas north' '24 paraganas south' 'adilabad' 'agar malwa' 'agra'
 'ahmadabad' 'ahmednagar' 'aizawl' 'ajmer' 'akola']

Salinity districts:
 ['south andamans' 'nicobars' 'north and middle andaman' 'anantapur'
 'chittoor' 'y.s.r.' 'east godavari' 'guntur' 'krishna' 'kurnool']


In [23]:
print(sugarcane_df.shape)
print(sugarcane_df.head())


(266, 15)
         crop  area1000hectares  production1000tonnes  \
8   Sugarcane             19.85                  3.59   
35  Sugarcane              0.72                  0.11   
64  Sugarcane              0.27                  0.05   
71  Sugarcane              7.57                  0.83   
95  Sugarcane              0.43                  0.09   

    irrigatedarea1000hectares  districtcode  year  statecode     statename  \
8                      19.820             1  2017         14  Chhattisgarh   
35                      0.616             2  2017         14  Chhattisgarh   
64                      0.267             3  2017         14  Chhattisgarh   
71                      7.484             4  2017         14  Chhattisgarh   
95                      0.431             5  2017         14  Chhattisgarh   

    nitrogenconsumptiontonnes  phosphateconsumptiontonnes  \
8                       91447                       53486   
35                      21279                       1258

In [24]:
sugarcane_df = pd.merge(sugarcane_df,rainfall_summary,on='districtname',how = 'left')

In [25]:
sugarcane_df = pd.merge(sugarcane_df,salinity_df,on= 'districtname',how='left')

In [26]:
pd.set_option('display.max_columns',None)

In [27]:
pd.set_option('display.width', None)

In [28]:
print(sugarcane_df.shape)

(266, 17)


In [29]:
sugarcane_df.head()

,crop,area1000hectares,production1000tonnes,irrigatedarea1000hectares,districtcode,year,statecode,statename,nitrogenconsumptiontonnes,phosphateconsumptiontonnes,potashconsumptiontonnes,districtname,irrigatedarea_capped,irrigatedarea_new,unirrigated_area,annual_rainfall,salinity_percent
0,Sugarcane,19.85,3.59,19.820,1,2017,14,Chhattisgarh,91447,53486,16993,durg,19.820,20.820,0.030,861.629998,0.000000
1,Sugarcane,0.72,0.11,0.616,2,2017,14,Chhattisgarh,21279,12584,4271,bastar,0.616,1.616,0.104,902.189990,0.000000
2,Sugarcane,0.27,0.05,0.267,3,2017,14,Chhattisgarh,92008,47721,15054,raipur,0.267,1.267,0.003,683.989988,0.000000
3,Sugarcane,7.57,0.83,7.484,4,2017,14,Chhattisgarh,72926,29713,6915,bilaspur,7.484,8.484,0.086,1857.730027,0.390813
4,Sugarcane,0.43,0.09,0.431,5,2017,14,Chhattisgarh,31282,13620,3707,raigarh,0.430,1.430,0.000,624.520000,0.000000
